In [1]:
import openai
import uuid
import time
import threading
from google.cloud import firestore

In [2]:
# openllm start NousResearch/Nous-Hermes-llama-2-7b --port 8001 --dtype float16 --backend vllm > openllm.log 2>&1 &

In [6]:
client = openai.OpenAI(
    api_key='EMPTY',
    base_url="http://localhost:8001/v1"
)
model = client.models.list().data[0].id

In [7]:
# Use the application default credentials.
db = firestore.Client()

In [8]:
collection_event = threading.Event()

def sort_messages_by_timestamp(chat):
    if 'messages' in chat and isinstance(chat['messages'], list):
        sorted_messages = sorted(chat['messages'], key=lambda x: x['timestamp'])
        return sorted_messages
    else:
        return chat['messages']  # or an appropriate response if the structure is not as expected

def clean_llm_output_message(message):
    for line in message.splitlines():
        if len(line) < 5:
            continue
        return line
    return message # message is already clean

def map_messages_to_openai_api(messages):
    openai_messages = []
    for message in messages:
        openai_messages.append({
            'content': message['message'],
            'role': message['role']
        })
    return openai_messages

def map_messages_to_alternating_roles(messages):
    alternating_messages = []
    last_role = None
    for message in messages:
        if message['role'] != last_role:
            alternating_messages.append(message)
            last_role = message['role']
        else:
            # If the last message was from the same role, append the message to the last message
            alternating_messages[-1]['message'] += '\n' + message['message']
    return alternating_messages

def on_snapshot(doc_snapshot, changes, read_time):
    for doc in doc_snapshot:
        # Get the last message from the chat
        print(f'Doc: {doc.id}')
        chat = doc.to_dict()
        sorted_chat_messages = sort_messages_by_timestamp(chat)
        # Check if the chat is empty
        if not sorted_chat_messages:
            print('Chat is empty')
            continue
        # Check for a message from the assistant
        if sorted_chat_messages[-1]['role'] == 'assistant':
            print('Last message is from the assistant')
            continue

        alternating_messages = map_messages_to_alternating_roles(sorted_chat_messages)
        openai_messages = map_messages_to_openai_api(alternating_messages)
        chat_completion = client.chat.completions.create(
            model=model,  # Replace with your preferred model
            messages=openai_messages
        )

        # Add the AI response to the chat
        cleaned_message = clean_llm_output_message(chat_completion.choices[0].message.content)
        print(f'AI Response: {cleaned_message}')
        db.collection('chats').document(doc.id).update({
            'messages': firestore.ArrayUnion([{
                'role': 'assistant',
                'message': cleaned_message,
                'timestamp': int(time.time() * 1000),
                'uuid': str(uuid.uuid4()),
                'senderUUID': 'test-model'
            }])
        })
    collection_event.set()

# Add the listener to your Firestore collection
chat_ref = db.collection('chats')
chat_watch = chat_ref.on_snapshot(on_snapshot)

Doc: 252a4016-b752-4ccb-a75a-821da1047704
Chat is empty
Doc: 33f6a81e-967e-4936-adf6-00b5eba77320
Chat is empty
Doc: 3ae3f4ef-4bf8-4576-a52e-237512fcfebc
Chat is empty
Doc: 48ca81f8-553f-4ea3-9042-eae10e7bd855
Last message is from the assistant
Doc: 60a8ac95-1480-47e3-a003-6bfd9682c131
Chat is empty
Doc: 779db376-4d0d-4f2b-acbc-a4654a23c5bd
Last message is from the assistant
Doc: a73a71ef-e89b-47dc-891a-affc483cb20e
Chat is empty
Doc: c384e952-42be-49e5-a8b0-694b072b1134
AI Response: My favorite cat breed is the Siamese cat. I find their sleek, colorpoint coat and piercing blue eyes very striking and elegant. They are also known for their playful and social personality, which makes them wonderful companions.
Doc: test-chat
Chat is empty
Doc: 252a4016-b752-4ccb-a75a-821da1047704
Chat is empty
Doc: 33f6a81e-967e-4936-adf6-00b5eba77320
Chat is empty
Doc: 3ae3f4ef-4bf8-4576-a52e-237512fcfebc
Chat is empty
Doc: 48ca81f8-553f-4ea3-9042-eae10e7bd855
Last message is from the assistant
Doc: 60a